## 根据2015赛季前三级联赛的转会数据绘制网络图

假设我现在只取2015赛季前三级联赛的的转会数据，先对*`clubs_league.csv`*进行一个简单统计
### 需要统计：

* 涉及哪些国家
* 总体的俱乐部数目-->Nodes
* 涉及联赛数目

In [1]:
import pandas as pd

In [14]:
df_clubs=pd.read_csv('clubs_league.csv')
# get the records of first 3 league level and season 2015
df_3lv_15=df_clubs[(df_clubs['season_id']==2015)&
                   (df_clubs['league_level'].isin(['First tier \xc2\xa0-','Second tier\xc2\xa0-','Third tier\xc2\xa0-']))]
# check the types of league level
# df_league_lv=df_clubs['league_level']
# print df_league_lv.unique()
print 'total clubs records: ', df_3lv_15.shape[0]

cnt_clubs_id=len(df_3lv_15['club_id'].unique())
# cnt_clubs_id2=len(set(df_3lv_15['club_id'].values.tolist()))
cnt_country_id=len(df_3lv_15['country_id'].unique())
cnt_clubs_name=len(df_3lv_15['club_name'].unique())
cnt_country_name=len(df_3lv_15['country_name'].unique())
cnt_league_id=len(df_3lv_15['league_id'].unique())
cnt_league_name=len(df_3lv_15['league_name'].unique())

print 'total club id No.: ',cnt_clubs_id
# print 'total club id No.: ',cnt_clubs_id2
print 'total club name No.: ',cnt_clubs_name
print 'total country id No.: ',cnt_country_id
print 'total country name No.: ',cnt_country_name
print 'total league id No.: ',cnt_league_id
print 'total league name No.: ',cnt_league_name
# save the the records of first 3 league level and season 2015
df_3lv_15.to_csv('clubs_3lv_15.csv',index=False)

total clubs records:  2583
total club id No.:  2335
total club name No.:  2334
total country id No.:  81
total country name No.:  81
total league id No.:  176
total league name No.:  166


### 记录筛选规则
* 严格前三级，即转会数据中*`move from`*和*`move to`*都必须是在2015赛季属于该国的前三级联赛的俱乐部
* 只取2015赛季的数据，从*`clubs_league.csv`*表获取2015赛季联赛级别在前三级的俱乐部集合 *`clubs_set`*
* 根据*`clubs_set`*来判断转会记录*`player_transfer_info_allClubs.csv`*中符合转会前后均在该集合中的记录，另外限制条件是赛季为*`15/16`*

In [23]:
df_transfers=pd.read_csv('player_transfer_info_allClubs.csv')
clubs_set=list(set(df_3lv_15['club_id'].values.tolist()))
# print len(clubs_set)
df_trans_lv3_15=df_transfers[(df_transfers['season']=='15/16')&
                             (df_transfers['move from id'].isin(clubs_set))&
                             (df_transfers['move to id'].isin(clubs_set))]
print 'total transfer records: ', df_trans_lv3_15.shape[0]
print 'total move from clubs(id): ',len(df_trans_lv3_15['move from id'].unique())
print 'total move from clubs(name): ',len(df_trans_lv3_15['move from'].unique())
print 'total move to clubs(id): ',len(df_trans_lv3_15['move to id'].unique())
print 'total move to clubs(name): ',len(df_trans_lv3_15['move to'].unique())
# separate the transfer records and load records according to the 'transfer fee' attribute
df_transfer=df_trans_lv3_15[(df_trans_lv3_15['transfer fee'] != 'Loan') & (df_trans_lv3_15['transfer fee'] != 'End of loan')]
df_loan=df_trans_lv3_15[(df_trans_lv3_15['transfer fee'] == 'Loan') | (df_trans_lv3_15['transfer fee'] == 'End of loan')]
df_loan.to_csv('player_loan_15.csv',index=False)   # 9523
df_transfer.to_csv('player_trans_15.csv',index=False)  # 15833
print '*************************************************'
print 'Transfer records in 2015: ',df_transfer.shape[0]
print 'Loan records in 2015: ',df_loan.shape[0]

total transfer records:  25356
total move from clubs(id):  2220
total move from clubs(name):  2219
total move to clubs(id):  2257
total move to clubs(name):  2256
*************************************************
Transfer records in 2015:  15833
Loan records in 2015:  9523


### 使用Gephi构建可视化网络
先简化一点

* Node table `clubs_3lv_15.csv`中的
 * club_id --> *Id*
 * club_name --> *Label*
 * country_id
 * country_name
* Edge table `player_trans_15.csv`中的
 * move from id --> *source*
 * move from
 * move to id --> *target*
 * move to
 * player id --> 边的属性

初步绘制图形：
* 按照节点的度对节点的*尺寸*进行放大/缩小（10~100）
* 按照国家对节点进行*颜色*区分（取数量最多的前十个国家上色）
* 使用*Layout*
 * Fruchterman Reingold
 * Force Atlas 2
得到如下原始图形

![player_trans_15_original.png](./player_trans_15_original.jpg)

接下来我试图根据第三张表*club_city_info.csv*来将俱乐部引射到对应的城市地图上看看分布效果和转会的情况，具体思路如下：

* 根据第三张表*club_city_info.csv*获取各个俱乐部所在的城市信息；
* 通过城市信息，使用第三方库*geocoder*计算各个俱乐部的经纬度并与*club_id*对应起来保存为新的属性*Latitude*和*Longitude*；
* 将属性*Latitude*和*Longitude*导入为原始结点表中的新的列
* 尝试利用Gephi上的第三方布局插件**Map of Countries**将原始分布转换为新的按照地理位置的分布
* 导出scg文件，尝试将分布图与地图叠加便于观察分析